In [0]:
## Code to install necessary libraries and perform authorization

"""
When you run this code, you will be asked to authenticate which can be done by 
clicking the click outputed and copy pasting the access code. Sometime you have to do this couple of time to gain access

If you ran this code after successful authentication during your session,
when asked for verification code without any link to browser, enter your google password
"""

!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 135004 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.14-0ubuntu1~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.14-0ubuntu1~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.14-0ubuntu1~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redir

In [0]:
## Mount google drive 
!mkdir -p drive
!google-drive-ocamlfuse -o nonempty drive
import sys
sys.path.insert(0, 'drive/bertTrain')

In [0]:
%cd drive

/content/drive


In [0]:
%cd bertTrain

/content/drive/bertTrain


In [0]:
!pwd

/content


In [0]:
!ls

In [0]:
!pip install sentencepiece
!git clone https://github.com/google-research/bert

import os
import sys
import json
import nltk
import random
import logging
import tensorflow as tf
import sentencepiece as spm

from glob import glob
from google.colab import auth, drive
from tensorflow.keras.utils import Progbar

sys.path.append("bert")

from bert import modeling, optimization, tokenization
from bert.run_pretraining import input_fn_builder, model_fn_builder

auth.authenticate_user()
  
# configure logging
log = logging.getLogger('tensorflow')
log.setLevel(logging.INFO)

# create formatter and add it to the handlers
formatter = logging.Formatter('%(asctime)s :  %(message)s')
sh = logging.StreamHandler()
sh.setLevel(logging.INFO)
sh.setFormatter(formatter)
log.handlers = [sh]

if 'COLAB_TPU_ADDR' in os.environ:
  log.info("Using TPU runtime")
  USE_TPU = True
  TPU_ADDRESS = 'grpc://' + os.environ['COLAB_TPU_ADDR']

  with tf.Session(TPU_ADDRESS) as session:
    log.info('TPU address is ' + TPU_ADDRESS)
    # Upload credentials to TPU.
    with open('/content/adc.json', 'r') as f:
      auth_info = json.load(f)
    tf.contrib.cloud.configure_gcs(session, credentials=auth_info)
    
else:
  log.warning('Not connected to TPU runtime')
  USE_TPU = False

     |████████████████████████████████| 1.0MB 3.5MB/s 
fatal: destination path 'bert' already exists and is not an empty directory.



2020-01-07 11:43:59,399 :  Using TPU runtime
2020-01-07 11:43:59,401 :  TPU address is grpc://10.118.0.250:8470


In [0]:
DEMO_MODE = False #@param {type:"boolean"}

if DEMO_MODE:
  CORPUS_SIZE = 1000000
else:
  CORPUS_SIZE = 100000000 #@param {type: "integer"}
               
!(head -n $CORPUS_SIZE oneMilData.txt) > subdataset.txt
!mv subdataset.txt oneMilData.txt

In [0]:
regex_tokenizer = nltk.RegexpTokenizer("\w+")

def normalize_text(text):
  # lowercase text
  text = str(text).lower()
  # remove non-UTF
  text = text.encode("utf-8", "ignore").decode()
  # remove punktuation symbols
  text = " ".join(regex_tokenizer.tokenize(text))
  return text

def count_lines(filename):
  count = 0
  with open(filename) as fi:
    for line in fi:
      count += 1
  return count

In [0]:
normalize_text('Şimdi o sevdiğim ışıklı yolu yalnız mı yürüyeceğim.')

'şimdi o sevdiğim ışıklı yolu yalnız mı yürüyeceğim'

In [0]:
RAW_DATA_FPATH = "rawDataset.txt" #@param {type: "string"}
PRC_DATA_FPATH = "clearDataset.txt" #@param {type: "string"}

# apply normalization to the dataset
# this will take a minute or two

#total_lines = count_lines(RAW_DATA_FPATH)
total_lines = count_lines(RAW_DATA_FPATH)
bar = Progbar(total_lines)

with open(RAW_DATA_FPATH,encoding="utf-8") as fi:
  with open(PRC_DATA_FPATH, "w",encoding="utf-8") as fo:
    for l in fi:
      fo.write(normalize_text(l)+"\n")
      bar.add(1)

1013628/1013628 [==============================] - 20s 20us/step


In [0]:
PRC_DATA_FPATH = "trainData.txt" #@param {type: "string"}

In [0]:
MODEL_PREFIX = "tokenizer" #@param {type: "string"}
VOC_SIZE = 128000 #@param {type:"integer"}
SUBSAMPLE_SIZE = 12800000 #@param {type:"integer"}
NUM_PLACEHOLDERS = 256 #@param {type:"integer"}

SPM_COMMAND = ('--input={} --model_prefix={} '
               '--hard_vocab_limit=false '
               '--vocab_size={} --input_sentence_size={} '
               '--shuffle_input_sentence=true '
               '--bos_id=-1 --eos_id=-1').format(
               PRC_DATA_FPATH, MODEL_PREFIX, #PRC_DATA_FPATH
               VOC_SIZE - NUM_PLACEHOLDERS, SUBSAMPLE_SIZE)
spm.SentencePieceTrainer.Train(SPM_COMMAND)

True

In [0]:
testcase = "Açıklamada depremin yerin 35 kilometre altında meydana geldiği belirtildi"

In [0]:
!head -n 100 tokenizer.vocab

<unk>	0
n	-4.09036
▁ve	-4.11379
̇	-4.17704
▁bir	-4.29645
▁i	-4.62051
a	-4.69107
▁bu	-4.78638
e	-4.85047
▁	-5.17623
▁da	-5.17971
▁de	-5.2368
▁için	-5.26406
▁ile	-5.33578
nin	-5.39393
nın	-5.43682
▁çok	-5.6713
ki	-5.79677
▁olarak	-5.80892
▁daha	-5.93603
▁en	-5.96653
▁ise	-6.02959
▁olan	-6.07376
dan	-6.08601
in	-6.12992
ın	-6.1627
den	-6.20758
ye	-6.24349
▁sonra	-6.2836
▁olduğunu	-6.32951
ya	-6.33094
yla	-6.33838
▁dedi	-6.35476
na	-6.3621
i	-6.3656
ne	-6.38275
▁kadar	-6.38307
▁ilk	-6.39438
▁bin	-6.39821
▁yeni	-6.41213
yle	-6.41348
▁son	-6.41356
▁var	-6.41458
▁büyük	-6.41884
▁yüzde	-6.45814
▁gibi	-6.46605
▁her	-6.48487
k	-6.4926
▁o	-6.51754
▁göre	-6.53287
▁ama	-6.54976
▁yıl	-6.55701
de	-6.56049
▁3	-6.56056
▁oldu	-6.56678
▁etti	-6.577
▁söyledi	-6.58069
▁ilgili	-6.5903
▁başkanı	-6.60447
▁2	-6.62096
▁olduğu	-6.62477
da	-6.65106
▁türkiye	-6.65858
▁yer	-6.68377
▁önce	-6.68752
▁türk	-6.68809
▁diye	-6.71508
▁devam	-6.75852
▁1	-6.76275
▁iyi	-6.76891
▁tarafından	-6.77167
▁milyon	-6.7777
▁ardından	-

In [0]:
def read_sentencepiece_vocab(filepath):
  voc = []
  with open(filepath, encoding='utf-8') as fi:
    for line in fi:
      voc.append(line.split("\t")[0])
  # skip the first <unk> token
  voc = voc[1:]
  return voc

 snt_vocab= read_sentencepiece_vocab("{}.vocab".format(MODEL_PREFIX))#MODEL_FREFIX isimli .vocab uzantılı dosyayı oku
print("Learnt vocab size: {}".format(len(snt_vocab)))
print("Sample tokens: {}".format(random.sample(snt_vocab, 10)))

Learnt vocab size: 127743
Sample tokens: ['▁tkdk', '▁kağıtlarında', 'mışsın', '▁viudez', '▁silahlar', '▁unutabilir', '▁ödemişten', 'mes', 'malısınız', '▁rifa']


In [0]:
def parse_sentencepiece_token(token):
    if token.startswith("▁"):
        return token[1:]
    else:
        return "##" + token
     

In [0]:
bert_vocab = list(map(parse_sentencepiece_token, snt_vocab))

In [0]:
ctrl_symbols = ["[PAD]","[UNK]","[CLS]","[SEP]","[MASK]"]
bert_vocab = ctrl_symbols + bert_vocab

In [0]:
bert_vocab += ["[UNUSED_{}]".format(i) for i in range(VOC_SIZE - len(bert_vocab))]
print(len(bert_vocab))

128000


In [0]:
VOC_FNAME = "vocab.txt" #@param {type:"string"}

with open(VOC_FNAME, "w") as fo:
  for token in bert_vocab:
    fo.write(str(token) + "\n")

In [0]:
!mkdir ./shards
!split -a 4 -l 256000 -d $PRC_DATA_FPATH ./shards/shard_ #PRC_DATA_FPATH
!ls ./shards/

shard_0000  shard_0001


In [0]:
MAX_SEQ_LENGTH = 128 #@param {type:"integer"}
MASKED_LM_PROB = 0.15 #@param
MAX_PREDICTIONS = 20 #@param {type:"integer"}
DO_LOWER_CASE = True #@param {type:"boolean"}
PROCESSES = 2 #@param {type:"integer"}
PRETRAINING_DIR = "pretraining_data" #@param {type:"string"}

In [0]:
XARGS_CMD = ("ls ./shards/ | "
             "xargs -n 1 -P {} -I{} "
             "python3 bert/create_pretraining_data.py "
             "--input_file=./shards/{} "
             "--output_file={}/{}.tfrecord "
             "--vocab_file={} "
             "--do_lower_case={} "
             "--max_predictions_per_seq={} "
             "--max_seq_length={} "
             "--masked_lm_prob={} "
             "--random_seed=34 "
             "--dupe_factor=5")

XARGS_CMD = XARGS_CMD.format(PROCESSES, '{}', '{}', PRETRAINING_DIR, '{}', 
                             VOC_FNAME, DO_LOWER_CASE, 
                             MAX_PREDICTIONS, MAX_SEQ_LENGTH, MASKED_LM_PROB)

In [0]:
tf.gfile.MkDir(PRETRAINING_DIR)
!$XARGS_CMD



W0106 16:01:05.038353 140482557745024 module_wrapper.py:139] From bert/create_pretraining_data.py:437: The name tf.logging.set_verbosity is deprecated. Please use tf.compat.v1.logging.set_verbosity instead.



W0106 16:01:05.038627 140482557745024 module_wrapper.py:139] From bert/create_pretraining_data.py:437: The name tf.logging.INFO is deprecated. Please use tf.compat.v1.logging.INFO instead.


W0106 16:01:05.038885 140482557745024 module_wrapper.py:139] From /content/drive/bertTrain/bert/tokenization.py:125: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.


W0106 16:01:05.039423 140048714598272 module_wrapper.py:139] From bert/create_pretraining_data.py:437: The name tf.logging.set_verbosity is deprecated. Please use tf.compat.v1.logging.set_verbosity instead.


W0106 16:01:05.039665 140048714598272 module_wrapper.py:139] From bert/create_pretraining_data.py:437: The name tf.logging.INFO is deprecated. Please use tf.compat.v1.logging.INFO instead.


W

In [0]:
BUCKET_NAME = "bert_500k_ikinci" #@param {type:"string"}
MODEL_DIR = "bert_model" #@param {type:"string"}
tf.gfile.MkDir(MODEL_DIR)
if not BUCKET_NAME:
  log.warning("WARNING: BUCKET_NAME is not set. "
              "You will not be able to train the model.")

In [0]:
# use this for BERT-base

bert_base_config = {
  "attention_probs_dropout_prob": 0.1, 
  "directionality": "bidi", 
  "hidden_act": "gelu", 
  "hidden_dropout_prob": 0.1, 
  "hidden_size": 768, 
  "initializer_range": 0.02, 
  "intermediate_size": 3072, 
  "max_position_embeddings": 512, 
  "num_attention_heads": 12, 
  "num_hidden_layers": 12, 
  "pooler_fc_size": 768, 
  "pooler_num_attention_heads": 12, 
  "pooler_num_fc_layers": 3, 
  "pooler_size_per_head": 128, 
  "pooler_type": "first_token_transform", 
  "type_vocab_size": 2, 
  "vocab_size": VOC_SIZE
}

with open("{}/bert_config.json".format(MODEL_DIR), "w") as fo:
  json.dump(bert_base_config, fo, indent=2)
  
with open("{}/{}".format(MODEL_DIR, VOC_FNAME), "w") as fo:
  for token in bert_vocab:
    fo.write(token+"\n")

In [0]:
if BUCKET_NAME:
  !gsutil -m cp -r $MODEL_DIR $PRETRAINING_DIR gs://$BUCKET_NAME

Copying file://bert_model/bert_config.json [Content-Type=application/json]...
Copying file://bert_model/vocab.txt [Content-Type=text/plain]...
Copying file://pretraining_data/shard_0000.tfrecord [Content-Type=application/octet-stream]...
==> NOTE: You are uploading one or more large file(s), which would run
significantly faster if you enable parallel composite uploads. This
feature can be enabled by editing the
"parallel_composite_upload_threshold" value in your .boto
configuration file. However, note that if you do this large files will
be uploaded as `composite objects
<https://cloud.google.com/storage/docs/composite-objects>`_,which
means that any user who downloads such objects will need to have a
compiled crcmod installed (see "gsutil help crcmod"). This is because
without a compiled crcmod, computing checksums on composite objects is
so slow that gsutil disables downloads of composite objects.

Copying file://pretraining_data/shard_0001.tfrecord [Content-Type=application/octet-st

In [0]:
BUCKET_NAME = "bert_500k_ikinci" #@param {type:"string"}
MODEL_DIR = "bert_model" #@param {type:"string"}
PRETRAINING_DIR = "pretraining_data" #@param {type:"string"}
VOC_FNAME = "vocab.txt" #@param {type:"string"}

# Input data pipeline config
TRAIN_BATCH_SIZE = 128 #@param {type:"integer"}
MAX_PREDICTIONS = 20 #@param {type:"integer"}
MAX_SEQ_LENGTH = 128 #@param {type:"integer"}
MASKED_LM_PROB = 0.15 #@param

# Training procedure config
EVAL_BATCH_SIZE = 64
LEARNING_RATE = 2e-5
TRAIN_STEPS = 500000 #@param {type:"integer"}
SAVE_CHECKPOINTS_STEPS = 2500 #@param {type:"integer"}
NUM_TPU_CORES = 8

if BUCKET_NAME:
  BUCKET_PATH = "gs://{}".format(BUCKET_NAME)
else:
  BUCKET_PATH = "."

BERT_GCS_DIR = "{}/{}".format(BUCKET_PATH, MODEL_DIR)
DATA_GCS_DIR = "{}/{}".format(BUCKET_PATH, PRETRAINING_DIR)

VOCAB_FILE = os.path.join(BERT_GCS_DIR, VOC_FNAME)
CONFIG_FILE = os.path.join(BERT_GCS_DIR, "bert_config.json")

INIT_CHECKPOINT = tf.train.latest_checkpoint(BERT_GCS_DIR)

bert_config = modeling.BertConfig.from_json_file(CONFIG_FILE)
input_files = tf.gfile.Glob(os.path.join(DATA_GCS_DIR,'*tfrecord'))

log.info("Using checkpoint: {}".format(INIT_CHECKPOINT))
log.info("Using {} data shards".format(len(input_files)))

2020-01-07 11:48:51,267 :  From /content/drive/bertTrain/bert/modeling.py:93: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.

2020-01-07 11:48:51,413 :  Using checkpoint: gs://bert_500k_ikinci/bert_model/model.ckpt-287500
2020-01-07 11:48:51,414 :  Using 2 data shards


In [0]:
model_fn = model_fn_builder(
      bert_config=bert_config,
      init_checkpoint=INIT_CHECKPOINT,
      learning_rate=LEARNING_RATE,
      num_train_steps=TRAIN_STEPS,
      num_warmup_steps=10,
      use_tpu=USE_TPU,
      use_one_hot_embeddings=True)

tpu_cluster_resolver = tf.contrib.cluster_resolver.TPUClusterResolver(TPU_ADDRESS)

run_config = tf.contrib.tpu.RunConfig(
    cluster=tpu_cluster_resolver,
    model_dir=BERT_GCS_DIR,
    save_checkpoints_steps=SAVE_CHECKPOINTS_STEPS,
    keep_checkpoint_max = 201,
    tpu_config=tf.contrib.tpu.TPUConfig(
        iterations_per_loop=SAVE_CHECKPOINTS_STEPS,
        num_shards=NUM_TPU_CORES,
        per_host_input_for_training=tf.contrib.tpu.InputPipelineConfig.PER_HOST_V2))

estimator = tf.contrib.tpu.TPUEstimator(
    use_tpu=USE_TPU,
    model_fn=model_fn,
    config=run_config,
    train_batch_size=TRAIN_BATCH_SIZE,
    eval_batch_size=EVAL_BATCH_SIZE)

train_input_fn = input_fn_builder(
        input_files=input_files,
        max_seq_length=MAX_SEQ_LENGTH,
        max_predictions_per_seq=MAX_PREDICTIONS,
        is_training=True)

2020-01-07 11:49:16,175 :  Estimator's model_fn (<function model_fn_builder.<locals>.model_fn at 0x7ff086236488>) includes params argument, but params are not passed to Estimator.
2020-01-07 11:49:16,177 :  Using config: {'_model_dir': 'gs://bert_500k_ikinci/bert_model', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 2500, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
cluster_def {
  job {
    name: "worker"
    tasks {
      key: 0
      value: "10.118.0.250:8470"
    }
  }
}
isolate_session_state: true
, '_keep_checkpoint_max': 201, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7ff086206898>

In [0]:
estimator.train(input_fn=train_input_fn, max_steps=TRAIN_STEPS)

2020-01-07 11:49:31,171 :  Querying Tensorflow master (grpc://10.118.0.250:8470) for TPU system metadata.
2020-01-07 11:49:31,188 :  Found TPU system:
2020-01-07 11:49:31,190 :  *** Num TPU Cores: 8
2020-01-07 11:49:31,191 :  *** Num TPU Workers: 1
2020-01-07 11:49:31,192 :  *** Num TPU Cores Per Worker: 8
2020-01-07 11:49:31,194 :  *** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:CPU:0, CPU, -1, 7486681392102846307)
2020-01-07 11:49:31,196 :  *** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 4709047913500348654)
2020-01-07 11:49:31,197 :  *** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:1, TPU, 17179869184, 13574518670160238463)
2020-01-07 11:49:31,198 :  *** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:2, TPU, 17179869184, 4680868200736624518)
2020-01-07 11:49:31,199 :  *** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:T

In [0]:
!!gsutil -m cp -r gs://$BUCKET_NAME/bert_model/ /content/drive/